<a href="https://colab.research.google.com/github/ipkrk1987/Machine-Learning/blob/master/StockPrediction_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.utils import plot_model

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA

In [ ]:

def reverse(data):
    data = data[::-1].reset_index(drop=True); 
    return data;
def mom_nextDay(df, n):  
    cl = df['Close']; 
    M = pd.Series((cl.pct_change(periods = n) >0).astype(int), name='Mom')
    df = df.join(M)
    return df
def momentum(df, n):  
    cl = df['Close']; 
    M = pd.Series(cl.pct_change(periods = n), name='Mom')
    df = df.join(M)
    return df
def moving_average(df, n,name,inverse=False,field='Close'):
    """Calculate the moving average for the given data.
    
    :param df: pandas.DataFrame
    :param n: 
    :return: pandas.DataFrame
    """
    if inverse:
        MA = pd.Series(df[field]/df[field].rolling(n, min_periods=n).mean(), name=name)
    else:
        MA = pd.Series(df[field].rolling(n, min_periods=n).mean(), name=name)
    df = df.join(MA)
    return df

def exponential_moving_average(df, n,name,inverse=False,field='Close'):
    """
    
    :param df: pandas.DataFrame
    :param n: 
    :return: pandas.DataFrame
    """
    
    #print(field)
    if inverse:
        EMA = pd.Series(1/df[field].ewm(span=n, min_periods=n).mean(), name=name)
    else:
        EMA = pd.Series(df[field].ewm(span=n, min_periods=n).mean(), name=name)
    df = df.join(EMA)
    return df


def weighted_moving_average(df, n,name,inverse=False,field='Close'):
    """
    
    :param df: pandas.DataFrame
    :param n: 
    :return: pandas.DataFrame
    """
    weights = np.arange(1,n+1,1)
    sum_weights = np.sum(weights)    
    
    if inverse:
        val = (df[field]
        .rolling(window=n, center=True)
        .apply(lambda x: sum_weights/np.sum(weights*x), raw=False)
        )
    else:
        val = (df[field]
        .rolling(window=n, center=True)
        .apply(lambda x: np.sum(weights*x) / sum_weights, raw=False)
        )
    
    
    WMA = pd.Series(val, name=name)
    df = df.join(WMA)
    return df

In [ ]:
def generateData(file, n=5, mas = [3,8,15,20,30,40,50,60,75,100,125,150,200]):
    stocks = pd.read_csv(file, skiprows=2,error_bad_lines=False);   
    
    stocks.columns = ['TICKER','DATE','OPEN','HIGH','LOW','Close','VOLUME']
    stocks = stocks.sort_values(by=['DATE'])    
    sr= stocks.reset_index(drop=True)
    s_ma = sr
    for ma in range(1, 201, 10):         
         s_ma = weighted_moving_average(s_ma,ma,"s_"+str(ma),True,'Close')
       
    print('sma generated')
    s_ma  = s_ma.dropna()    
    s_mamo = momentum(s_ma,n)    
    s_mamo.loc[:,'Mom']  = s_mamo.loc[:,'Mom'].shift(-n)
    s_mamo = s_mamo.iloc[n+5:]
    sr = s_mamo
    sr_null = sr[sr.Mom.isna()]
    sr = sr.dropna()    
    sr.loc[:,'buy'] = (sr.Mom > 0).astype('int')    
    return sr,sr_null

In [ ]:


def testAdaBoost(file,days):
    steps = [('clf', AdaBoostClassifier())]
    pipe = Pipeline(steps) # define the pipeline object.
    etc  = ExtraTreeClassifier()
    dtc  = DecisionTreeClassifier()   
    nb   = GaussianNB()
    parameteres = {'clf__n_estimators':[10],'clf__random_state':[0],'clf__base_estimator':[etc,dtc]}
    grid = GridSearchCV(pipe, param_grid=parameteres, cv=3)   
    sr,sr_null = generateData(file,days)   
    pca_df, pca_dfnull = getPCAData(sr,sr_null)
    Y = pca_df.buy
    X_train, X_test, y_train, y_test = train_test_split(pca_df, Y, test_size=0.1, random_state=42,stratify = Y)
    grid.fit(X_train,y_train)    
    Ypred_test = grid.predict(X_test);
    y_current = grid.predict(pca_dfnull)   
    score_grid = accuracy_score(y_test,Ypred_test)#grid.score(dftest,Ytest)
    return score_grid,y_current,sr_null.tail(1)


def testMLP(file,days):
    steps = [('clf', MLPClassifier())]
    pipe = Pipeline(steps) # define the pipeline object. 
   
    parameteres = {'clf__hidden_layer_sizes':[(100,1), (100,2), (100,3)],'clf__activation':[ 'logistic']}
    grid = GridSearchCV(pipe, param_grid=parameteres, cv=3) 
   
    sr,sr_null = generateData(file,days)   
    pca_df, pca_dfnull = getPCAData(sr,sr_null)
    Y = pca_df.buy
    X_train, X_test, y_train, y_test = train_test_split(pca_df, Y, test_size=0.1, random_state=42,stratify = Y)
    grid.fit(X_train,y_train)    
    Ypred_test = grid.predict(X_test);
    y_current = grid.predict(pca_dfnull)   
    score_grid = accuracy_score(y_test,Ypred_test)#grid.score(dftest,Ytest)
    print(grid.best_params_)
    return score_grid,y_current,sr_null.tail(1)

def testRandomForest(file,days):     
    grid = RandomForestClassifier()   
    return testClassifier(file,days,grid)

def testXGBooster(file,days):
    grid =  xgb.XGBClassifier(objective="binary:logistic", random_state=42)
    return testClassifier(file,days,grid)


def testPerceptron(file,days):    
    grid = Perceptron()   
    return testClassifier(file,days,grid)


def testClassifier(file,days,classifier):    
    grid = classifier   
    sr,sr_null = generateData(file,days)   
    pca_df, pca_dfnull = getPCAData(sr,sr_null)
    Y = sr.buy    
    
    
    
    ## Split data in half.
    X_train, X_test, y_train, y_test = train_test_split(pca_df, Y, test_size=0.8, random_state=42,stratify = Y)
    grid.fit(X_train,y_train)     
    
     
    
    y_train_pred = grid.predict(X_train)
    y_train_proba = grid.predict_proba(X_train) 
   
    
    y_null_proba = grid.predict_proba(pca_dfnull)    
    y_null_pred = grid.predict(pca_dfnull) 
    
    
    #= grid.predict(X_test)
    y_test_proba = grid.predict_proba(X_test)
    y_test_pred = grid.predict(X_test) 
    
    
#     cnew2 = xgb.XGBClassifier(objective="binary:logistic", random_state=42) 
#     cnew2 = trainProbabilites(y_train_proba,y_train,cnew2)
#     y_test_pred = cnew2.predict(y_test_proba)
    
    #### This is used to compute the accuracy and cm for normal algorithm###
    acc = accuracy_score(y_test,y_test_pred)
    cm = confusion_matrix(y_test,y_test_pred)
    cr = classification_report(y_test,y_test_pred)
    f1 = f1_score(y_test,y_test_pred)
    

         
     
    
    
    #add probabilites of classes.
    sr_null[grid.classes_] = pd.DataFrame(y_null_proba[-days:], index=sr_null.index) 
    ## Add last week values to the newdata to be predicted.
    sr_null[['PC']] = pd.DataFrame(sr.Close.tail(days).values, index=sr_null.index)     
    sr_null[['Pred']] = pd.DataFrame(y_null_pred, index=sr_null.index)
    return f1,acc,cm,cr,sr_null,sr, y_null_proba,y_null_pred,y_test_proba,y_train_proba,y_test,y_train


def trainProbabilites(proba_train,y_train,classifier):
    classifier.fit(proba_train,y_train)    
    return classifier

In [ ]:
def getPCAData(sr,sr_null):
    imputer = KNNImputer(n_neighbors=3)
    #imputer.fit_transform(X)
    ##### Get PCA DATA FOR STOCK DATA WITH MOM.
    
    df  = sr.loc[:, sr.columns.str.startswith('s_')]    
    scaler = MinMaxScaler()     
    pca = PCA() # create a PCA object
    pca.fit(df) # do the math
    pca_data = pca.transform(df) # get PCA coordinates for scaled_data
    per_var = np.round(pca.explained_variance_ratio_* 100, decimals=1)
    labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]
    #the following code makes a fancy looking plot using PC1 and PC2
    pca_df = pd.DataFrame(pca_data, columns=labels)   
    x = df.var(axis=1)        
    pca_df = pca_df.join(pd.Series(x,name='x')) 
    X = imputer.fit_transform(pca_df)         
    pca_df = pd.DataFrame(X)
    
    ##### Get PCA DATA FOR STOCK DATA WITH NO MOM.
    
    df_null  = sr_null.loc[:, sr_null.columns.str.startswith('s_')] 
    df_null = scaler.fit_transform(df_null)  
    pca_data_null = pca.transform(df_null) # get PCA coordinates for scaled_data    
    pca_df_null = pd.DataFrame(pca_data_null, columns=labels)   
    x_null = df_null.var(axis=1)
    pca_df_null = pca_df_null.join(pd.Series(x_null,name='x'))
    X_null = imputer.fit_transform(pca_df_null)         
    pca_df_null = pd.DataFrame(X_null)
   
    
    return pca_df,pca_df_null

Build the model



In [ ]:
import os
directory = os.path.join("../content/stockdata")
print(directory)
days = 20
scores = []
for root,dirs,files in os.walk(directory):
    for f in files:
        if f.endswith(".csv"):
            fn = os.path.join(directory,f) 
            try:     
              print(fn) 
              sr,sr_null = generateData(fn,days)               
              pca_df, pca_dfnull = getPCAData(sr,sr_null)
              print(pca_df.shape)
              Y = sr.buy.values

              onehot_encoder=OneHotEncoder(sparse=False)
              reshaped=Y.reshape(len(Y), 1)
              y_onehot=onehot_encoder.fit_transform(reshaped)
             
              train_x, test_x, train_y, test_y = train_test_split(pca_df, y_onehot, test_size=0.9, random_state=42,stratify = Y)
              model = Sequential()
              model.add(Dense(12, input_dim=21, activation='relu'))
              model.add(Dense(2, activation='softmax'))
              model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

              # Test on unseen data
              model.fit(train_x, train_y, verbose=0, batch_size=5, epochs=200)
              print(model.summary())
              results = model.evaluate(test_x, test_y)

              print('Final test set loss: {:4f}'.format(results[0]))
              print('Final test set accuracy: {:4f}'.format(results[1]))  

             
            except Exception as inst:
              print(inst)
              #print("Exception occured",file)
             

**Print the model**

In [ ]:

#!pip install keras_sequential_ascii
from keras_sequential_ascii import keras2ascii
keras2ascii(model)